# Vault Transfers

*Work in Progress*

This notebook aims to provide a brief demonstration of how this codebase could be used for visualize vault transfers.

In [ ]:
"""load dependencies and define constants"""
import sys

import pandas as pd
from decimal import Decimal
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
from enum import Enum

sys.path.append("..")
from src.yearn import Network, Yearn, Subgraph
from src.utils.web3 import Web3Provider 
from src.utils.network import client, parse_json

# constants
ZERO_ADDRESS = '0x0000000000000000000000000000000000000000'
FROM_BLOCK = 13900000  # Dec-29-2021
BATCH_SIZE = 500000
API_ENDPOINT = "https://api.etherscan.io/api"

class Stats(Enum):
    # top-level stats
    vaults = "vaults"
    overall_account_transfers = "overall_account_transfers"
    # sub stats
    count = "count"
    shares = "shares"
    shares_usdc = "shares_usdc"
    account_transfers = "account_transfers"

    def __repr__(self):
        return self.value

pd.set_option("expand_frame_repr", False)

In [ ]:
"""get current block, load vaults from the yearn instance and initialize subgraph instance"""
load_dotenv()
w3 = Web3Provider(Network.Mainnet)
current_block = w3.provider.eth.get_block("latest")["number"]
print(f"current block number: {current_block}")

yearn = Yearn(Network.Mainnet)
vaults = yearn.vaults
print(f"loaded metadata for {len(vaults)} vaults (v2)")

subgraph = Subgraph(Network.Mainnet)

In [ ]:
"""get vaults, accounts and overall accounts' stats for withdrawal"""
withdrawals_stats = {Stats.vaults: {}, Stats.overall_account_transfers: {}}
for idx, vault in enumerate(vaults):
    # approximate the share price by the current usdc value, skip if not available
    try:
        share_price = w3.get_usdc_price(vault.token.address)
    except:
        continue

    transfers = subgraph.transfers("withdrawals", vault, from_block=FROM_BLOCK, to_block=current_block)

    # current vault's stats
    _dict = {
        Stats.count: transfers.count,
        Stats.shares: transfers.shares,
        Stats.shares_usdc: float(Decimal(transfers.shares) * share_price),
    }
    print(f"vault: {vault.address} {dict((k, v) for k, v in _dict.items() if k in _dict)}")

    account_items = transfers.account_transfers.items()
    sorted_account_transfers = sorted(account_items, key=lambda item: item[1].shares, reverse=True)
    account_transfers = {}
    
    # loop current vault's account_transfers
    for index, (account, transfer) in enumerate(sorted_account_transfers):
        # current vault's single account_transfers stat
        _inner_dict = {
            Stats.count: transfer.count,
            Stats.shares: transfer.shares,
            Stats.shares_usdc: float(Decimal(transfer.shares) * share_price)
        }
        account_transfers[account] = _inner_dict

        # print top 10 accounts' stats for current vault
        if index < 10:
            print(f"{index}: {account} {_inner_dict}")

        # add overall_account_transfers to withdrawals stats
        overall_account_transfers = withdrawals_stats[Stats.overall_account_transfers]
        if overall_account_transfers.get(account) is None:
            overall_account_transfers[account] = {}
        account_stats = overall_account_transfers[account]
        overall_account_transfers[account] = {k: account_stats.get(k, 0) + _inner_dict[k] for k in _inner_dict}

    # add vault's stats to withdrawals stats
    withdrawals_stats[Stats.vaults][vault.address] = {**_dict, Stats.account_transfers: account_transfers}
    print()

In [ ]:
# withdrawals stats for all vaults and overall_account_transfers
for key in withdrawals_stats.keys():
    vault_stats = []
    for subkey, subval in withdrawals_stats[key].items():
        # only include address, count, shares and shares_usdc
        new_val = dict((k, v) for k, v in subval.items() if k in _dict.keys())
        vault_stats.append((subkey, *new_val.values()))
    df = pd.DataFrame(vault_stats, columns=["address", "count", "shares", "approx. USDC value"])
    df["approx. USDC value"] = pd.to_numeric(df["approx. USDC value"])
    df.set_index(keys=["address"], inplace=True)
    df.sort_values("approx. USDC value", inplace=True, ascending=False)
    print(f"top withdrawal {key} by approx. USDC value")
    print(df.head(10))
    print()

In [ ]:
"""get vaults, accounts and overall accounts' stats for deposits"""
deposits_stats = {Stats.vaults: {}, Stats.overall_account_transfers: {}}
for idx, vault in enumerate(vaults):
    # approximate the share price by the current usdc value, skip if not available
    try:
        share_price = w3.get_usdc_price(vault.token.address)
    except:
        continue
    
    transfers = subgraph.transfers("deposits", vault, from_block=FROM_BLOCK, to_block=current_block)

    # current vault's stats
    _dict = {
        Stats.count: transfers.count,
        Stats.shares: transfers.shares,
        Stats.shares_usdc: float(Decimal(transfers.shares) * share_price)
    }
    print(f"vault: {vault.address} {dict((k, v) for k, v in _dict.items() if k in _dict)}")

    account_items = transfers.account_transfers.items()
    sorted_account_transfers = sorted(account_items, key=lambda item: item[1].shares, reverse=True)
    account_transfers = {}
    
    # loop current vault's account_transfers
    for index, (account, transfer) in enumerate(sorted_account_transfers):
        # current vault's single account_transfers stat
        _inner_dict = {
            Stats.count: transfer.count,
            Stats.shares: transfer.shares,
            Stats.shares_usdc: float(Decimal(transfer.shares) * share_price)
        }
        account_transfers[account] = _inner_dict

        # print top 10 accounts' stats for current vault
        if index < 10:
            print(f"{index}: {account} {_inner_dict}")

        # add overall_account_transfers to deposits stats
        overall_account_transfers = deposits_stats[Stats.overall_account_transfers]
        if overall_account_transfers.get(account) is None:
            overall_account_transfers[account] = {}
        account_stats = overall_account_transfers[account]
        overall_account_transfers[account] = {k: account_stats.get(k, 0) + _inner_dict[k] for k in _inner_dict}

    # add vault's stats to deposits stats
    deposits_stats[Stats.vaults][vault.address] = {**_dict, Stats.account_transfers: account_transfers}
    print()

In [ ]:
# deposits stats for all vaults and overall_account_transfers
for key in deposits_stats.keys():
    vault_stats = []
    for subkey, subval in deposits_stats[key].items():
        # only include address, count, shares and shares_usdc
        new_val = dict((k, v) for k, v in subval.items() if k in _dict.keys())
        vault_stats.append((subkey, *new_val.values()))
    df = pd.DataFrame(vault_stats, columns=["address", "count", "shares", "approx. USDC value"])
    df["approx. USDC value"] = pd.to_numeric(df["approx. USDC value"])
    df.set_index(keys=["address"], inplace=True)
    df.sort_values("approx. USDC value", inplace=True, ascending=False)
    print(f"top deposit {key} by approx. USDC value")
    print(df.head(10))
    print()

In [ ]:
"""helper functions to calculate net transfers"""

# Calculate net transfers of these keys
keys = [Stats.count, Stats.shares, Stats.shares_usdc]

def equalize_lists(*args: list[tuple]) -> None:
    """Make all lists equal in size"""
    items = [*args]
    max_length = 0
    for item in items:
        max_length = max(max_length, len(item))
    for item in items:
        item += [(None, None)] * (max_length - len(item))

def get_net_transfer(withdrawals_items: list[tuple[str, dict]], deposits_items: list[tuple[str, dict]]) -> dict[str, dict[str, float]]:
    """Get net transfer of vaults and overall account transfers"""
    net_transfer = {}

    for (withdraw_address, withdraw_stats), (deposit_address, deposit_stats) in zip(withdrawals_items, deposits_items):
        if withdraw_address is not None:
            if net_transfer.get(withdraw_address) is None:
                net_transfer[withdraw_address] = {}
            _dict = {}
            for k in keys:
                if k == Stats.count:
                    _dict[k] = net_transfer[withdraw_address].get(k, 0) + withdraw_stats[k]
                else:
                    _dict[k] = net_transfer[withdraw_address].get(k, 0) - withdraw_stats[k]
            net_transfer[withdraw_address] = _dict
        if deposit_address is not None:
            if net_transfer.get(deposit_address) is None:
                net_transfer[deposit_address] = {}
            net_transfer[deposit_address] = {k: net_transfer[deposit_address].get(k, 0) + deposit_stats[k] for k in keys}

    return net_transfer

In [ ]:
"""net transfer of each vault's transfer count, shares and shares usdc"""
withdrawals_vaults = list(withdrawals_stats[Stats.vaults].items())
deposits_vaults = list(deposits_stats[Stats.vaults].items())
equalize_lists(withdrawals_vaults, deposits_vaults)
net_vaults = get_net_transfer(withdrawals_vaults, deposits_vaults)

# TODO: WIP
# withdrawals stats for all vaults and overall_account_transfers
# vault_stats = []
# for subkey, subval in net_vaults.items():
#     # only include address, count, shares and shares_usdc
#     new_val = dict((k, v) for k, v in subval.items() if k in _dict.keys())
#     vault_stats.append((subkey, *new_val.values()))
# df = pd.DataFrame(vault_stats, columns=["address", "count", "shares", "approx. USDC value"])
# df["approx. USDC value"] = pd.to_numeric(df["approx. USDC value"])
# df.set_index(keys=["address"], inplace=True)
# df.sort_values("approx. USDC value", inplace=True, ascending=False)
# print(f"top withdrawal vaults by approx. USDC value")
# print(df.head(10))
# print()

In [ ]:
"""net transfer of each account's transfer count, shares and shares usdc scoped by all vaults"""
withdrawals_account_transfers = list(withdrawals_stats[Stats.overall_account_transfers].items())
deposits_account_transfers = list(deposits_stats[Stats.overall_account_transfers].items())
equalize_lists(withdrawals_account_transfers, deposits_account_transfers)
net_overall_account_transfers = get_net_transfer(withdrawals_account_transfers, deposits_account_transfers)

In [ ]:
"""net transfer of each vault's account's transfer count, shares and shares usdc scoped by each vault"""
net_transfer = {}
for (withdraw_address, withdraw_stats), (deposit_address, deposit_stats) in zip(withdrawals_vaults, deposits_vaults):
    if withdraw_address is not None:
        if net_transfer.get(withdraw_address) is None:
            net_transfer[withdraw_address] = {}

        for (account, account_transfer) in list(withdraw_stats[Stats.account_transfers].items()):
            if net_transfer[withdraw_address].get(account) is None:
                net_transfer[withdraw_address][account] = {}
            _dict = {}
            for k in keys:
                if k == Stats.count:
                    _dict[k] = net_transfer[withdraw_address][account].get(k, 0) + account_transfer[k]
                else:
                    _dict[k] = net_transfer[withdraw_address][account].get(k, 0) - account_transfer[k]
            net_transfer[withdraw_address][account] = _dict
        
    if deposit_address is not None:
        if net_transfer.get(deposit_address) is None:
            net_transfer[deposit_address] = {}

        for (account, account_transfer) in list(deposit_stats[Stats.account_transfers].items()):
            if net_transfer[deposit_address].get(account) is None:
                net_transfer[deposit_address][account] = {}
            account_val = {k: net_transfer[deposit_address][account].get(k, 0) + account_transfer[k] for k in keys}
            net_transfer[deposit_address][account] = account_val